In [29]:
import dotenv

dotenv.load_dotenv()

True

## Assemble RAG Pipeline

In [31]:
from rag import weaviate_utils
from llama_index import VectorStoreIndex

# assumes that the database has already been populated
weaviate_client = weaviate_utils.get_weaviate_client()
vector_store = weaviate_utils.as_vector_store(weaviate_client, class_name="DocsChunk")
index = VectorStoreIndex.from_vector_store(vector_store)

In [32]:
query_engine = index.as_query_engine(verbose=True)

## Evaluation

In [34]:
from trulens_eval import Tru
tru = Tru()

In [35]:
from trulens_eval.feedback.provider.openai import OpenAI
openai = OpenAI()

In [36]:
from trulens_eval.app import App

# select context to be used in feedback
context = App.select_context(query_engine)

In [46]:
from trulens_eval.feedback import Groundedness, Feedback

grounded = Groundedness(groundedness_provider=OpenAI())
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [47]:
# response relevance
f_qa_relevance = Feedback(openai.relevance, name="Response Relevance").on_input_output()

✅ In Response Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Response Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [48]:
import numpy as np

# context relevance
f_qs_relevance = (
    Feedback(openai.qs_relevance, name="Context Relevance")
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [55]:
from trulens_eval import TruLlama

tru_query_engine_recorder = TruLlama(
    app=query_engine,
    app_id="basic-1",
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance]
)

In [56]:
with tru_query_engine_recorder as recording:
    query_engine.query("How can I turn off the ECS controller?")

In [51]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.178.44:8501


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [57]:
tru.stop_dashboard()

In [54]:
tru.reset_database()